In [1]:
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import numpy as np
from scipy.misc import imread, imresize
import scipy.io
import pdb
import string
import h5py
from nltk.tokenize import word_tokenize
import json

/home/nitin/anaconda3/envs/san/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import re
def tokenize(sentence):
    return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])", sentence) if i!='' and i!=' ' and i!='\n'];

In [3]:
string = 'Hi, My name is Nitin'
tokenize(string)

['Hi', ',', 'My', 'name', 'is', 'Nitin']

In [4]:
word_tokenize(string) #NLTK

['Hi', ',', 'My', 'name', 'is', 'Nitin']

In [ ]:
def prepro_question(imgs, params):
    print ('example processed tokens:')
    for i,img in enumerate(imgs):
        #print(i)
        #print(img'\n')
        s = img['question']
        #print(s)
        if params['token_method'] == 'nltk':
            txt = word_tokenize(str(s).lower())
            #print(txt'\n')
        else:
            txt = tokenize(s)
            #print(txt'\n')
        img['processed_tokens'] = txt
        if i < 10: print (txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done)   \r" %  (i, len(imgs), i*100.0/len(imgs)) )
            sys.stdout.flush()   
    return imgs

Short snippets corresponding to `prepro_question()`:

In [5]:
import json

with open('data/vqa_raw_train.json') as js:
    imgs_train = json.load(js)
    #imgs_train = json.load(open('input_train_json', 'r'))

In [ ]:
for i, img in enumerate(imgs_train):
    s = img['question']
    #print(i, s)
    print(s)

```
What is this photo taken looking through?
What position is this man playing?
What color is the players shirt?
Is this man a professional baseball player?
.
.
.
.
```

In [ ]:
for i, img in enumerate(imgs_train):
    s = img['question']
    txt = word_tokenize(str(s).lower())
    print(txt)

```
['what', 'is', 'this', 'photo', 'taken', 'looking', 'through', '?']
['what', 'position', 'is', 'this', 'man', 'playing', '?']
['what', 'color', 'is', 'the', 'players', 'shirt', '?']
['is', 'this', 'man', 'a', 'professional', 'baseball', 'player', '?']
['what', 'color', 'is', 'the', 'snow', '?']
['what', 'is', 'the', 'person', 'doing', '?']
['what', 'color', 'is', 'the', 'persons', 'headwear', '?']
.
.
.
```

In [ ]:
for i, img in enumerate(imgs_train):
    print("processing %d/%d (%.2f%% done)   \r" %  (i, len(imgs_train), i*100.0/len(imgs_train)) )

```
processing 0/443757 (0.00% done)   
processing 1/443757 (0.00% done)   
processing 2/443757 (0.00% done)   
processing 3/443757 (0.00% done)   
processing 4/443757 (0.00% done)   
processing 5/443757 (0.00% done)   
processing 6/443757 (0.00% done)   
processing 7/443757 (0.00% done)   
processing 8/443757 (0.00% done)   
processing 9/443757 (0.00% done)   
processing 10/443757 (0.00% done)
.
.
.
```

In [ ]:
def build_vocab_question(imgs, params):
    # build vocabulary for question and answers.

    count_thr = params['word_count_threshold']

    # count up the number of words
    counts = {}
    for img in imgs:
        for w in img['processed_tokens']:
            counts[w] = counts.get(w, 0) + 1
    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
    print ('top words and their counts:')
    print ('\n'.join(map(str,cw[:20])))

    # print some stats
    total_words = sum(counts.values())
    print ('total words:', total_words)
    bad_words = [w for w,n in counts.items() if n <= count_thr]
    vocab = [w for w,n in counts.items() if n > count_thr]
    bad_count = sum(counts[w] for w in bad_words)
    print ('number of bad words: %d/%d = %.2f%%' % (len(bad_words), len(counts), len(bad_words)*100.0/len(counts)))
    print ('number of words in vocab would be %d' % (len(vocab), ))
    print ('number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count*100.0/total_words))


    # lets now produce the final annotation
    # additional special UNK token we will use below to map infrequent words to
    print ('inserting the special UNK token')
    vocab.append('UNK')
  
    for img in imgs:
        txt = img['processed_tokens']
        question = [w if counts.get(w,0) > count_thr else 'UNK' for w in txt]
        img['final_question'] = question

    return imgs, vocab

Corrsponding short snippets for `build_vocab_question()`:

In [6]:
# Counts the number of words in the tokens

counts = {}
#for img in imgs:   
#    for w in txt:
#        counts[w] = counts.get(w, 0) + 1
        
for i, img in enumerate(imgs_train):
    s = img['question']
    txt = word_tokenize(str(s).lower())
    
    img['processed_tokens'] = txt
    for w in txt:
        counts[w] = counts.get(w, 0) + 1
       # print('%s : %s' % (w, counts[w]))

```
many : 14450
roads : 22
does : 6645
this : 25464
have : 3471
? : 130628
is : 81764
the : 95072
hillside : 12
steep : 10
? : 130629
do : 2983
they : 2276
.
.
.
```

In [12]:
# sort the words in order to get the top words

cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
print ('Top words and their counts:')
print ('\n'.join(map(str, cw[:5])))

Top words and their counts:
(443931, '?')
(322836, 'the')
(277570, 'is')
(185348, 'what')
(104094, 'are')


In [13]:
total_words = sum(counts.values())
print ('total words:', total_words)

total words: 3216813


In [14]:
count_thr = 0 # > 0 to see changes
bad_words = [w for w,n in counts.items() if n <= count_thr]
vocab = [w for w,n in counts.items() if n > count_thr]
bad_count = sum(counts[w] for w in bad_words)

In [15]:
print ('number of bad words: %d/%d = %.2f%%' % (len(bad_words), len(counts), len(bad_words)*100.0/len(counts)))
print ('number of words in vocab would be %d' % (len(vocab), ))
print ('number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count*100.0/total_words))

number of bad words: 0/13771 = 0.00%
number of words in vocab would be 13771
number of UNKs: 0/3216813 = 0.00%


In [16]:
itow = {i+1:w for i,w in enumerate(vocab)} # a 1-indexed vocab translation table
wtoi = {w:i+1 for i,w in enumerate(vocab)} # inverse table

In [ ]:
print(itow)

In [ ]:
print(wtoi)

In [29]:
json.dump(wtoi, open('word_to_ix_test.json', 'w'))

UNK : used to replace the rare words that did not fit in the vocab. Eg: Sentence `My name is guotong1988` will be translated into `My name is _unk_`.

In [ ]:
print ('inserting the special UNK token')
vocab.append('UNK')

for img in imgs_train:
    txt = word_tokenize(str(s).lower())
    img['processed_tokens'] = txt
    
    question = [w if counts.get(w,0) > count_thr else 'UNK' for w in img['processed_tokens']]
    img['final_question'] = question
    
    #for w in img['processed_tokens']:
    #    if counts.get(w, 0) > count_thr:
    #        question = w
    #    else:
    #        question = 'UNK'
            
    #img['final_question'] = question
    
    print(img)

```
inserting the special UNK token

{'ques_id': 458752000, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What is this photo taken looking through?', 'ans': 'net', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}

{'ques_id': 458752001, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What position is this man playing?', 'ans': 'pitcher', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}

{'ques_id': 458752002, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What color is the players shirt?', 'ans': 'orange', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}

{'ques_id': 458752003, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'Is this man a professional baseball player?', 'ans': 'yes', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}

{'ques_id': 262146000, 'img_path': 'train2014/COCO_train2014_000000262146.jpg', 'question': 'What color is the snow?', 'ans': 'white', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}

{'ques_id': 262146001, 'img_path': 'train2014/COCO_train2014_000000262146.jpg', 'question': 'What is the person doing?', 'ans': 'skiing', 'processed_tokens': ['is', 'that', 'a', 'laptop', '?'], 'final_question': ['is', 'that', 'a', 'laptop', '?']}
```

In [ ]:
def get_top_answers(imgs, params):
    counts = {}
    for img in imgs:
        ans = img['ans'] 
        counts[ans] = counts.get(ans, 0) + 1

    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
    print ('top answer and their counts:')    
    print ('\n'.join(map(str,cw[:20])))
    
    vocab = []
    for i in range(params['num_ans']):
        vocab.append(cw[i][1])

    return vocab[:params['num_ans']]

Corrsponding short snippets for `get_top_answers()`:

In [ ]:
counts = {}
for img in imgs_train:
    ans = img['ans'] 
    counts[ans] = counts.get(ans, 0) + 1
    print("%s : %s" %(ans, counts[ans]))

```
net : 1
pitcher : 1
orange : 1
yes : 1
white : 1
skiing : 1
red : 1
frisbee : 1
yes : 2
frisbee : 2
yes : 3
yes : 4
contrail : 1
yes : 5
white and purple : 1
brushing teeth : 1
yes : 6
no : 1
.
.
.
```

In [17]:
cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
print ('top answer and their counts:')    
print ('\n'.join(map(str,cw[:5])))

top answer and their counts:
(443931, '?')
(322836, 'the')
(277570, 'is')
(185348, 'what')
(104094, 'are')


In [18]:
top_ans = vocab

In [19]:
vocab = []
num_ans = 1000 # As supplied at the command line
for i in range(num_ans):
    vocab.append(cw[i][1])
    print(vocab[i])
#print(vocab)

?
the
is
what
are
this
in
on
of
a
how
many
color
there
man
does
people
picture
's
to
wearing
it
these
where
person
have
kind
or
photo
do
you
woman
doing
type
animal
they
room
be
holding
animals
can
for
dog
cat
at
train
his
he
sign
that
which
any
food
shirt
bus
water
see
an
why
made
playing
plane
sitting
shown
plate
table
right
with
time
left
boy
taken
sport
background
was
number
standing
pizza
girl
white
look
who
from
being
has
her
wall
street
all
and
sky
she
hand
day
ground
top
men
say
visible
looking
red
clock
eating
building
would
truck
image
child
like
here
front
bear
up
toilet
name
going
out
one
behind
bird
game
same
car
fruit
green
riding
horse
seen
light
trees
black
object
scene
giraffe
their
bed
hair
hat
colors
brand
elephant
two
vehicle
cake
pictured
ball
side
floor
flowers
blue
could
glasses
bike
did
head
been
lights
used
grass
bathroom
boat
window
baby
someone
open
giraffes
sunny
player
horses
old
cars
next
computer
bench
down
shoes
elephants
meal
birds
umbrella
,
back
zebra

In [ ]:
def encode_question(imgs, params, wtoi):

    max_length = params['max_length']
    N = len(imgs)

    label_arrays = np.zeros((N, max_length), dtype='uint32')
    label_length = np.zeros(N, dtype='uint32')
    question_id = np.zeros(N, dtype='uint32')
    question_counter = 0
    for i,img in enumerate(imgs):
        question_id[question_counter] = img['ques_id']
        label_length[question_counter] = min(max_length, len(img['final_question'])) # record the length of this sequence
        question_counter += 1
        for k,w in enumerate(img['final_question']):
            if k < max_length:
                label_arrays[i,k] = wtoi[w]
    
    return label_arrays, label_length, question_id


Corrsponding short snippets for `encode_question()`:

In [41]:
max_length = 26
N = len(imgs_train)
print(N)

443757


In [42]:
label_arrays = np.zeros((N, max_length), dtype='uint32')
label_length = np.zeros(N, dtype='uint32')
question_id = np.zeros(N, dtype='uint32')
question_counter = 0

In [43]:
print(label_arrays)
print(label_length)
print(question_id)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [ ]:
for i,img in enumerate(imgs_train):
    question_id[question_counter] = img['ques_id']
    label_length[question_counter] = min(max_length, len(img['final_question'])) # record the length of this sequence
    question_counter += 1
    for k,w in enumerate(img['final_question']):
        if k < max_length:
            label_arrays[i,k] = wtoi[w]
            #print(label_arrays)

In [ ]:
def encode_answer(imgs, atoi):
    N = len(imgs)
    ans_arrays = np.zeros(N, dtype='uint32')

    for i, img in enumerate(imgs):
        ans_arrays[i] = atoi[img['ans']]

    return ans_arrays


Corrsponding short snippets for `encode_answer()`:

In [20]:
N = len(imgs_train)
ans_array = np.zeros(N, dtype='uint32')

In [21]:
atoi = {w:i+1 for i,w in enumerate(top_ans)}
itoa = {i+1:w for i,w in enumerate(top_ans)}
json.dump(itoa, open('ix_to_ans.json', 'w'))

In [22]:
for i, img in enumerate(imgs_train):
    ans_arrays[i] = atoi[img['ans']]
    #print(ans_array[i])
    
print(ans_array)

NameError: name 'ans_arrays' is not defined

In [ ]:
def encode_mc_answer(imgs, atoi):
    N = len(imgs)
    mc_ans_arrays = np.zeros((N, 18), dtype='uint32')

    for i, img in enumerate(imgs):
        for j, ans in enumerate(img['MC_ans']):
            mc_ans_arrays[i,j] = atoi.get(ans, 0)
    return mc_ans_arrays

Corrsponding short snippets for `encode__mc_answer()`:

In [68]:
N = len(imgs_train)
mc_ans_arrays = np.zeros((N, 18), dtype='uint32')

In [ ]:
for i, img in enumerate(imgs_train):
    for j, ans in enumerate(img['MC_ans']):
        mc_ans_arrays[i,j] = atoi.get(ans, 0)

In [ ]:
def filter_question(imgs, atoi):
    new_imgs = []
    for i, img in enumerate(imgs):
        if atoi.get(img['ans'],len(atoi)+1) != len(atoi)+1:
            new_imgs.append(img)

    print ('question number reduce from %d to %d '%(len(imgs), len(new_imgs)))
    return new_imgs

Corrsponding short snippets for `filter_question()`:

In [ ]:
new_imgs = []

for i, img in enumerate(imgs_train):
    if atoi.get(img['ans'],len(atoi)+1) != len(atoi)+1:
        new_imgs.append(img)
        #print(new_imgs[i])
print(new_imgs)

```
[
{'ques_id': 458752000, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What is this photo taken looking through?', 'ans': 'net', 'processed_tokens': ['what', 'is', 'this', 'photo', 'taken', 'looking', 'through', '?']},

{'ques_id': 458752001, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What position is this man playing?', 'ans': 'pitcher', 'processed_tokens': ['what', 'position', 'is', 'this', 'man', 'playing', '?']}, 

{'ques_id': 458752002, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'What color is the players shirt?', 'ans': 'orange', 'processed_tokens': ['what', 'color', 'is', 'the', 'players', 'shirt', '?']}, 

{'ques_id': 458752003, 'img_path': 'train2014/COCO_train2014_000000458752.jpg', 'question': 'Is this man a professional baseball player?', 'ans': 'yes', 'processed_tokens': ['is', 'this', 'man', 'a', 'professional', 'baseball', 'player', '?']},

{'ques_id': 262146000, 'img_path': 'train2014/COCO_train2014_000000262146.jpg', 'question': 'What color is the snow?', 'ans': 'white', 'processed_tokens': ['what', 'color', 'is', 'the', 'snow', '?']},

{'ques_id': 262146001, 'img_path': 'train2014/COCO_train2014_000000262146.jpg', 'question': 'What is the person doing?', 'ans': 'skiing', 'processed_tokens': ['what', 'is', 'the', 'person', 'doing', '?']},

{'ques_id': 262146002, 'img_path': 'train2014/COCO_train2014_000000262146.jpg', 'question': 'What color is the persons headwear?', 'ans': 'red', 'processed_tokens': ['what', 'color', 'is', 'the', 'persons', 'headwear', '?']}, 

{'ques_id': 524291000, 'img_path': 'train2014/COCO_train2014_000000524291.jpg', 'question': "What is in the person's hand?", 'ans': 'frisbee', 'processed_tokens': ['what', 'is', 'in', 'the', 'person', "'s", 'hand', '?']},

{'ques_id': 524291001, 'img_path': 'train2014/COCO_train2014_000000524291.jpg', 'question': 'Is the dog waiting?', 'ans': 'yes', 'processed_tokens': ['is', 'the', 'dog', 'waiting', '?']},
```

In [25]:
print ('question number reduce from %d to %d '%(len(imgs_train), len(new_imgs)))

question number reduce from 443757 to 401152 


In [ ]:
def get_unqiue_img(imgs):
    count_img = {}
    N = len(imgs)
    img_pos = np.zeros(N, dtype='uint32')
    for img in imgs:
        count_img[img['img_path']] = count_img.get(img['img_path'], 0) + 1

    unique_img = [w for w,n in count_img.items()]
    imgtoi = {w:i+1 for i,w in enumerate(unique_img)} # add one for torch, since torch start from 1.


    for i, img in enumerate(imgs):
        img_pos[i] = imgtoi.get(img['img_path'])

    return unique_img, img_pos

Corrsponding short snippets for `get_unique_img()`:

In [27]:
count_img = {}
N = len(imgs_train)
img_pos = np.zeros(N, dtype='uint32')

In [ ]:
for img in imgs_train:
    count_img[img['img_path']] = count_img.get(img['img_path'], 0) + 1
    
print(count_img)

```
{'train2014/COCO_train2014_000000458752.jpg': 4,
'train2014/COCO_train2014_000000262146.jpg': 3,
'train2014/COCO_train2014_000000524291.jpg': 3,
'train2014/COCO_train2014_000000393221.jpg': 3,
'train2014/COCO_train2014_000000393223.jpg': 4,
'train2014/COCO_train2014_000000393224.jpg': 6,
'train2014/COCO_train2014_000000524297.jpg': 4,
'train2014/COCO_train2014_000000393227.jpg': 5,
'train2014/COCO_train2014_000000131084.jpg': 3,
'train2014/COCO_train2014_000000131074.jpg': 6,
'train2014/COCO_train2014_000000393230.jpg': 11,
'train2014/COCO_train2014_000000262187.jpg': 4, 
```

In [29]:
unique_img = [w for w,n in count_img.items()]
imgtoi = {w:i+1 for i,w in enumerate(unique_img)} # add one for torch, since torch start from 1.

In [ ]:
print(unique_img)
#print(imgtoi)

```
['train2014/COCO_train2014_000000458752.jpg', 'train2014/COCO_train2014_000000262146.jpg', 'train2014/COCO_train2014_000000524291.jpg', 'train2014/COCO_train2014_000000393221.jpg', 'train2014/COCO_train2014_000000393223.jpg', 'train2014/COCO_train2014_000000393224.jpg', 'train2014/COCO_train2014_000000524297.jpg', 'train2014/COCO_train2014_000000393227.jpg', 'train2014/COCO_train2014_000000131084.jpg', 'train2014/COCO_train2014_000000131074.jpg', 'train2014/COCO_train2014_000000393230.jpg', 'train2014/COCO_train2014_000000131087.jpg', 'train2014/COCO_train2014_000000131075.jpg', 'train2014/COCO_train2014_000000137045.jpg', 'train2014/COCO_train2014_000000131093.jpg', 'train2014/COCO_train2014_000000524311.jpg', 'train2014/COCO_train2014_000000000025.jpg', 'train2014/COCO_train2014_000000524314.jpg', 'train2014/COCO_train2014_000000262171.jpg', 'train2014/COCO_train2014_000000262172.jpg', 'train2014/COCO_train2014_000000131101.jpg', 'train2014/COCO_train2014_000000000030.jpg', 'train2014/COCO_train2014_000000524320.jpg', 'train2014/COCO_train2014_000000240304.jpg', 'train2014/COCO_train2014_000000000034.jpg', 'train2014/COCO_train2014_000000393251.jpg', 'train2014/COCO_train2014_000000262180.jpg', 'train2014/COCO_train2014_000000524325.jpg', 'train2014/COCO_train2014_000000043697.jpg', 'train2014/COCO_train2014_000000262184.jpg', 'train2014/COCO_train2014_000000131113.jpg', 'train2014/COCO_train2014_000000262187.jpg', 'train2014/COCO_train2014_000000131118.jpg', 'train2014/COCO_train2014_000000262191.jpg', 'train2014/COCO_train2014_000000000049.jpg', 'train2014/COCO_train2014_000000524338.jpg', 'train2014/COCO_train2014_000000213863.jpg', 
```

In [ ]:
print(imgtoi)

```
{'train2014/COCO_train2014_000000458752.jpg': 1, 'train2014/COCO_train2014_000000262146.jpg': 2, 'train2014/COCO_train2014_000000524291.jpg': 3, 'train2014/COCO_train2014_000000393221.jpg': 4, 'train2014/COCO_train2014_000000393223.jpg': 5, 'train2014/COCO_train2014_000000393224.jpg': 6, 'train2014/COCO_train2014_000000524297.jpg': 7, 'train2014/COCO_train2014_000000393227.jpg': 8, 'train2014/COCO_train2014_000000131084.jpg': 9, 'train2014/COCO_train2014_000000131074.jpg': 10, 'train2014/COCO_train2014_000000393230.jpg': 11, 'train2014/COCO_train2014_000000131087.jpg': 12, 'train2014/COCO_train2014_000000131075.jpg': 13, 'train2014/COCO_train2014_000000137045.jpg': 14, 'train2014/COCO_train2014_000000131093.jpg': 15, 'train2014/COCO_train2014_000000524311.jpg': 16, 'train2014/COCO_train2014_000000000025.jpg': 17, 'train2014/COCO_train2014_000000524314.jpg': 18, 'train2014/COCO_train2014_000000262171.jpg': 19, 'train2014/COCO_train2014_000000262172.jpg': 20, 'train2014/COCO_train2014_000000131101.jpg': 21, 'train2014/COCO_train2014_000000000030.jpg': 22, 'train2014/COCO_train2014_000000524320.jpg': 23, 'train2014/COCO_train2014_000000240304.jpg': 24, 'train2014/COCO_train2014_000000000034.jpg': 25, 'train2014/COCO_train2014_000000393251.jpg': 26, 'train2014/COCO_train2014_000000262180.jpg': 27, 'train2014/COCO_train2014_000000524325.jpg': 28, 'train2014/COCO_train2014_000000043697.jpg': 29, 'train2014/COCO_train2014_000000262184.jpg': 30, 'train2014/COCO_train2014_000000131113.jpg': 31, 'train2014/COCO_train2014_000000262187.jpg': 32, 'train2014/COCO_train2014_000000131118.jpg': 33, 'train2014/COCO_train2014_000000262191.jpg': 34
```

In [35]:
for i, img in enumerate(imgs_train):
    img_pos[i] = imgtoi.get(img['img_path'])
    #print(img_pos[i])
    
print(img_pos)

[    1     1     1 ... 82783 82783 82783]
